# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [3]:
df_raw = pd.read_csv("admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [11]:
# frequency table for prestige and whether or not someone was admitted
print(df_raw.groupby('prestige').count())
print
print(df_raw.groupby('prestige').mean())

          admit  gre  gpa
prestige                 
1.0          61   61   61
2.0         150  148  149
3.0         121  121  121
4.0          67   67   67

             admit         gre       gpa
prestige                                
1.0       0.540984  611.803279  3.453115
2.0       0.353333  596.621622  3.364027
3.0       0.231405  574.876033  3.432893
4.0       0.179104  570.149254  3.318358


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [12]:
df_dums = pd.get_dummies(df_raw, columns=['prestige'])

In [17]:
df_dums.columns = ["admit", "gre", "gpa", 
                   "prestige_1", "prestige_2", "prestige_3", "prestige_4"]

### 2.2 When modeling our class variables, how many do we need? 

Answer: The classic answer is that we need 1 fewer class variables than there are classes. The excluded class becomes the reference class. However, a fuller response is that the number of class variables needed depends on the extent of colinearity between the class variables. 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [18]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(df_dums.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           0           1           0
1      1  660.0  3.67           0           0           1           0
2      1  800.0  4.00           1           0           0           0
3      1  640.0  3.19           0           0           0           1
4      0  520.0  2.93           0           0           0           1


In [27]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(df_dums['admit'], df_dums['prestige_1'], rownames=['admit'], colnames=['prestige'])

prestige,0,1
admit,,
0,245,28
1,94,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

The odds ration is calculated as follows:

$$ \Psi  = \frac{admit\; vs\; fail\; when\; prestige=1}{admit\; vs\; fail\; when\; prestige =0}  = \frac{\frac{X=1\; when\; Y=1}{X=1\; when\; Y=0}}{\frac{X=0\; when\; Y=1}{X=0\; when\; Y=0}}      $$

In this particular case:

$$ \Psi = \dfrac{\frac{33}{94}} {\frac{28}{245}} = 3.07$$

In [40]:
#check with scipy
import scipy.stats as stats
arr = pd.crosstab(df_dums['admit'], df_dums['prestige_1']).values
oddsratio, pvalue = stats.fisher_exact(arr)
print("OddsR: ", oddsratio, "p-Value:", pvalue)

('OddsR: ', 3.0718085106382977, 'p-Value:', 8.6407727167175569e-05)


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [57]:
#by hand
(28/245.0)/(33/94.0)

0.3255411255411255

In [66]:
#with scipy
stats.fisher_exact(np.flip(arr, axis = 1))

(0.32554112554112552, 8.6407727167175569e-05)

#### 3.3 Calculate the odds ratio

In [47]:
float(33/94)

0.0

#### 3.4 Write this finding in a sentenance: 

Attending a number 1 ranked college increases the probability of being admitted to graduate school. This result is significant at 99%. 

#### 3.5 Print the cross tab for prestige_4

In [67]:
pd.crosstab(df_dums['admit'], df_dums['prestige_4'], rownames=['admit'], colnames=['prestige'])

prestige,0,1
admit,,
0,218,55
1,115,12


#### 3.6 Calculate the OR 

In [69]:
arr4 = pd.crosstab(df_dums['admit'], df_dums['prestige_4']).values
oddsratio, pvalue = stats.fisher_exact(arr4)
print("OddsR: ", oddsratio, "p-Value:", pvalue)

('OddsR: ', 0.41359683794466401, 'p-Value:', 0.0091652849547235182)


#### 3.7 Write this finding in a sentence

Answer: The probability of gaining admission to grad school is negtively affected by having attended a college with prestige of 4. This result is singificant at 99%

## Part 4. Analysis

In [70]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(df_dums.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           1           0
1      1  660.0  3.67           0           1           0
2      1  800.0  4.00           0           0           0
3      1  640.0  3.19           0           0           1
4      0  520.0  2.93           0           0           1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [71]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [77]:
train_cols = ['intercept', "gre", "gpa", "prestige_2", "prestige_3", "prestige_4"]

#### 4.2 Fit the model

In [78]:
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [79]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 14 Aug 2017   Pseudo R-squ.:                 0.08166
Time:                        18:00:21   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
intercept     -3.8769      1.142     -3.393      0.001        -6.116    -1.638
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
==============================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [80]:
np.exp(result.params)

intercept     0.020716
gre           1.002221
gpa           2.180027
prestige_2    0.506548
prestige_3    0.262192
prestige_4    0.211525
dtype: float64

In [93]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                2.5%     97.5%        OR
intercept   0.002207  0.194440  0.020716
gre         1.000074  1.004372  1.002221
gpa         1.136120  4.183113  2.180027
prestige_2  0.272168  0.942767  0.506548
prestige_3  0.133377  0.515419  0.262192
prestige_4  0.093329  0.479411  0.211525


#### 4.5 Interpret the OR of Prestige_2

The estimate on prestige_2 is not terribly precise. At the 97.5th percentile of confidence the odds ratio is almost 1, which would indicate no relationship between being from a 2nd prestige school and admission. At the 2.5th percentile of confidence there is a negative relationship between the probability of admission and being from a second prestige college. 

#### 4.6 Interpret the OR of GPA

Answer: GRE alone does not seem to affect the probability of admission. GPA seems to be postiviely related to probability of admission although the estimate is fairly imprecise. 

## Part 5: Predicted probablities


## I AM NOT CLEAR ON THIS PART, I WILL RETURN TO IT AFTER CLASS

As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [ ]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [ ]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

#### 5.1 Recreate the dummy variables

In [ ]:
# recreate the dummy variables

# keep only what we need for making predictions


#### 5.2 Make predictions on the enumerated dataset

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.